In [1]:
import FinanceDataReader as fdr

## 1. 상장종목 리스트 가져오기

- FinanceDataReader을 이용하여 종목 조회
- 상장폐지 종목은 리스트에서 제외

In [2]:
stocks = fdr.StockListing('KRX').dropna()
stocks

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
7581,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)",2015-08-07,12월,"박철범, 오길영",http://www.hyungkuk.com,서울특별시
7582,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,임규준,http://www.insurance.co.kr,서울특별시
7585,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
7586,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시


## 2. 종목코드 및 종목명 저장

- stocks에서 종목 코드, 종목명만 추출

In [3]:
import time
import pandas as pd

In [4]:
stocks[['Symbol', 'Name']]

,Symbol,Name
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS홀딩스
4,265520,AP시스템
...,...,...
7581,189980,흥국에프엔비
7582,000540,흥국화재
7585,003280,흥아해운
7586,037440,희림


In [5]:
stock_list = stocks[['Symbol', 'Name']]
stock_list.columns = ['종목코드', '종목명']

In [6]:
stock_list

,종목코드,종목명
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS홀딩스
4,265520,AP시스템
...,...,...
7581,189980,흥국에프엔비
7582,000540,흥국화재
7585,003280,흥아해운
7586,037440,희림


## 3. 손익계산서 수집

- https://comp.fnguide.com 의 포괄손익계산서의 데이터 수집 함수 생성

In [7]:
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.151 Whale/3.14.134.62 Safari/537.36"
}

In [9]:
# 손익계산서 수집 함수
def getIS(stock, rpt_type, freq):
    if rpt_type.upper()=='CONSOLIDATED': # 연결 연간/분기 손익 계산서
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701".format(stock[0])
    else: # 별도 연간/분기 손익계산서
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=B&NewMenuID=103&stkGb=701".format(stock[0])
    
    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')
    num_col = 4 # 최근 4개 데이터
    
    if freq.upper() == 'A': # 연간 손익계산서 영역
        is_a = soup.find(id='divSonikY')    
    else: # 분기 손익계산서 영역
        is_a = soup.find(id='divSonikQ')
    is_a = is_a.find_all(['tr'])
    
    items_kr = [is_a[m].find(['th']).get_text().replace('\n', '').replace('\xa0', '').replace('계산에 참여한 계정 펼치기', '')
               for m in range(1, len(is_a))]
    
    period = [is_a[0].find_all('th')[n].get_text() for n in range(1, num_col+1)]
    globals()['period'] = period
        
    for item, i in zip(items_kr, range(1, len(is_a))):
        temps = []
        for j in range(0, num_col):
            temp = [float(is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                   if is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != '' \
                   else (0 if is_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0' \
                        else 0)]
            temps.append(temp[0])
            
        globals()[item] = temps
    
    if rpt_type.upper() == 'CONSOLIDATED':
        pass
    else: # 별도 연간 손익계산서 해당 항목을 NULL값으로 채움
        globals()['지배주주순이익'], globals()['비지배주주순이익'] = [np.NaN]*num_col, [np.NaN]*num_col
    if freq.upper() == 'A': # 연긴
        is_domestic = pd.DataFrame({
            '종목코드': stock[0],
            '종목명': stock[1],
            '기간': period,
            '매출액': 매출액,
            '매출원가': 매출원가,
            '매출총이익': 매출총이익,
            '판매비': 판매비,
            '관리비': 관리비,
            '영업이익': 영업이익,
            '금융수익': 금융수익,
            '금융원가': 금융원가,
            '기타수익': 기타수익,
            '기타비용': 기타비용,
            '관계기업관련손익': globals()['종속기업,공동지배기업및관계기업관련손익'],
            'EBIT': 세전계속사업이익,
            '법인세비용': 법인세비용,
            '계속영업이익': 계속영업이익,
            '중단영업이익': 중단영업이익,
            '당기순이익': 당기순이익,
            '지배주주순이익': globals()['지배주주순이익'],
            '비지배주주순이익': globals()['비지배주주순이익']
        }, index=period)
    else: # 분기
        is_domestic = pd.DataFrame({
            '종목코드': stock[0],
            '종목명': stock[1],
            '기간': period,
            '매출액': 매출액,
            '매출원가': 매출원가,
            '매출총이익': 매출총이익,
            '판매비': 판매비,
            '관리비': 관리비,
            '영업이익': 영업이익,
            '금융수익': 금융수익,
            '금융원가': 금융원가,
            '기타수익': 기타수익,
            '기타비용': 기타비용,
            '관계기업관련손익': globals()['종속기업,공동지배기업및관계기업관련손익'],
            'EBIT': 세전계속사업이익,
            '법인세비용': 법인세비용,
            '계속영업이익': 계속영업이익,
            '중단영업이익': 중단영업이익,
            '당기순이익': 당기순이익
        }, index=period)
    return is_domestic # DataFrame 생성

## 4. 연결, 연간 손익계산 데이터 수집 및 DB 저장
- stock_con_a 데이터 프레임에 저장
- consolidated, A

In [10]:
import pymysql
import sqlalchemy
import time, json, requests
from datetime import datetime, timedelta, date
from sqlalchemy import create_engine

In [11]:
server = '127.0.0.1'
user = 'root'
passwd = 'abcd123456'
db = 'krx_stock_data'

# sqlalchemy의 create_engine을 이용하여 DB 연결
engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8'.format(user, passwd, server, db))

In [13]:
stock_list

,종목코드,종목명
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS홀딩스
4,265520,AP시스템
...,...,...
7581,189980,흥국에프엔비
7582,000540,흥국화재
7585,003280,흥아해운
7586,037440,희림


In [21]:
list = stock_list.values.tolist() # stock_list의 데이터를 list형태로 변환
print(len(stock_list))
list

2336


[['060310', '3S'],
 ['095570', 'AJ네트웍스'],
 ['006840', 'AK홀딩스'],
 ['054620', 'APS홀딩스'],
 ['265520', 'AP시스템'],
 ['211270', 'AP위성'],
 ['027410', 'BGF'],
 ['282330', 'BGF리테일'],
 ['032790', 'BNGT'],
 ['138930', 'BNK금융지주'],
 ['001460', 'BYC'],
 ['013720', 'CBI'],
 ['001040', 'CJ'],
 ['079160', 'CJ CGV'],
 ['035760', 'CJ ENM'],
 ['311690', 'CJ 바이오사이언스'],
 ['000120', 'CJ대한통운'],
 ['011150', 'CJ씨푸드'],
 ['097950', 'CJ제일제당'],
 ['051500', 'CJ프레시웨이'],
 ['058820', 'CMG제약'],
 ['023460', 'CNH'],
 ['056730', 'CNT85'],
 ['065770', 'CS'],
 ['083660', 'CSA 코스믹'],
 ['000590', 'CS홀딩스'],
 ['012030', 'DB'],
 ['016610', 'DB금융투자'],
 ['005830', 'DB손해보험'],
 ['000990', 'DB하이텍'],
 ['139130', 'DGB금융지주'],
 ['001530', 'DI동일'],
 ['000210', 'DL'],
 ['001880', 'DL건설'],
 ['068790', 'DMS'],
 ['004840', 'DRB동일'],
 ['241520', 'DSC인베스트먼트'],
 ['155660', 'DSR'],
 ['069730', 'DSR제강'],
 ['017940', 'E1'],
 ['245620', 'EDGC'],
 ['037370', 'EG'],
 ['278990', 'EMB'],
 ['365550', 'ESR켄달스퀘어리츠'],
 ['050120', 'ES큐브'],
 ['084180', 'EV수성'],

- 연결_연간 데이터

In [ ]:
for stock in list:
    getIS(stock, 'consolidated', 'A').to_sql(name='stock_con_a',con=engine,if_exists='append',index=False,
             dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.
                '종목코드': sqlalchemy.types.VARCHAR(10),
                '종목명': sqlalchemy.types.TEXT(),
                '기간': sqlalchemy.types.TEXT(),
                '매출액': sqlalchemy.types.FLOAT(),
                '매출원가': sqlalchemy.types.FLOAT(),
                '매출총이익': sqlalchemy.types.FLOAT(),
                '판매비': sqlalchemy.types.FLOAT(),
                '관리비': sqlalchemy.types.FLOAT(),
                '영업이익': sqlalchemy.types.FLOAT(),
                '금융수익': sqlalchemy.types.FLOAT(),
                '금융원가': sqlalchemy.types.FLOAT(),
                '기타수익': sqlalchemy.types.FLOAT(),
                '기타비용': sqlalchemy.types.FLOAT(),
                '관계기업관련손익': sqlalchemy.types.FLOAT(),
                'EBIT': sqlalchemy.types.FLOAT(),
                '법인세비용': sqlalchemy.types.FLOAT(),
                '계속영업이익': sqlalchemy.types.FLOAT(),
                '중단영업이익': sqlalchemy.types.FLOAT(),
                '당기순이익': sqlalchemy.types.FLOAT(),
                '지배주주순이익': sqlalchemy.types.FLOAT(),
                '비지배주주순이익': sqlalchemy.types.FLOAT()
             })

- 연결_분기 데이터

In [ ]:
for stock in list:
    getIS(stock, 'consolidated', 'Q').to_sql(name='stock_con_q',con=engine,if_exists='append',index=False,
             dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.
                '종목코드': sqlalchemy.types.VARCHAR(10),
                '종목명': sqlalchemy.types.TEXT(),
                '기간': sqlalchemy.types.TEXT(),
                '매출액': sqlalchemy.types.FLOAT(),
                '매출원가': sqlalchemy.types.FLOAT(),
                '매출총이익': sqlalchemy.types.FLOAT(),
                '판매비': sqlalchemy.types.FLOAT(),
                '관리비': sqlalchemy.types.FLOAT(),
                '영업이익': sqlalchemy.types.FLOAT(),
                '금융수익': sqlalchemy.types.FLOAT(),
                '금융원가': sqlalchemy.types.FLOAT(),
                '기타수익': sqlalchemy.types.FLOAT(),
                '기타비용': sqlalchemy.types.FLOAT(),
                '관계기업관련손익': sqlalchemy.types.FLOAT(),
                'EBIT': sqlalchemy.types.FLOAT(),
                '법인세비용': sqlalchemy.types.FLOAT(),
                '계속영업이익': sqlalchemy.types.FLOAT(),
                '중단영업이익': sqlalchemy.types.FLOAT(),
                '당기순이익': sqlalchemy.types.FLOAT(),
             })

- 별도_연간 데이터

In [15]:
for stock in list:
    getIS(stock, 'unconsolidated', 'A').to_sql(name='stock_uncon_aa',con=engine,if_exists='append',index=False,
             dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.
                '종목코드': sqlalchemy.types.VARCHAR(10),
                '종목명': sqlalchemy.types.TEXT(),
                '기간': sqlalchemy.types.TEXT(),
                '매출액': sqlalchemy.types.FLOAT(),
                '매출원가': sqlalchemy.types.FLOAT(),
                '매출총이익': sqlalchemy.types.FLOAT(),
                '판매비': sqlalchemy.types.FLOAT(),
                '관리비': sqlalchemy.types.FLOAT(),
                '영업이익': sqlalchemy.types.FLOAT(),
                '금융수익': sqlalchemy.types.FLOAT(),
                '금융원가': sqlalchemy.types.FLOAT(),
                '기타수익': sqlalchemy.types.FLOAT(),
                '기타비용': sqlalchemy.types.FLOAT(),
                '관계기업관련손익': sqlalchemy.types.FLOAT(),
                'EBIT': sqlalchemy.types.FLOAT(),
                '법인세비용': sqlalchemy.types.FLOAT(),
                '계속영업이익': sqlalchemy.types.FLOAT(),
                '중단영업이익': sqlalchemy.types.FLOAT(),
                '당기순이익': sqlalchemy.types.FLOAT(),
                '지배주주순이익': sqlalchemy.types.FLOAT(),
                '비지배주주순이익': sqlalchemy.types.FLOAT()
             })

- 별도_분기 데이터

In [ ]:
for stock in list:
    getIS(stock, 'unconsolidated', 'Q').to_sql(name='stock_uncon_q',con=engine,if_exists='append',index=False,
             dtype = { # sql에 저장할 때, 데이터 유형도 설정할 수 있다.
                '종목코드': sqlalchemy.types.VARCHAR(10),
                '종목명': sqlalchemy.types.TEXT(),
                '기간': sqlalchemy.types.TEXT(),
                '매출액': sqlalchemy.types.FLOAT(),
                '매출원가': sqlalchemy.types.FLOAT(),
                '매출총이익': sqlalchemy.types.FLOAT(),
                '판매비': sqlalchemy.types.FLOAT(),
                '관리비': sqlalchemy.types.FLOAT(),
                '영업이익': sqlalchemy.types.FLOAT(),
                '금융수익': sqlalchemy.types.FLOAT(),
                '금융원가': sqlalchemy.types.FLOAT(),
                '기타수익': sqlalchemy.types.FLOAT(),
                '기타비용': sqlalchemy.types.FLOAT(),
                '관계기업관련손익': sqlalchemy.types.FLOAT(),
                'EBIT': sqlalchemy.types.FLOAT(),
                '법인세비용': sqlalchemy.types.FLOAT(),
                '계속영업이익': sqlalchemy.types.FLOAT(),
                '중단영업이익': sqlalchemy.types.FLOAT(),
                '당기순이익': sqlalchemy.types.FLOAT(),
             })

##  5. DB 저장 확인

In [16]:
## 연결_연간 데이터
sql = 'select * from stock_con_a'
db_data = pd.read_sql(sql, engine)
db_data

,종목코드,종목명,기간,매출액,매출원가,매출총이익,판매비,관리비,영업이익,금융수익,기타수익,기타비용,관계기업관련손익,EBIT,법인세비용,계속영업이익,중단영업이익,당기순이익,지배주주순이익,비지배주주순이익
0,060310,3S,2019/03,357.52,304.38,53.14,7.15,12.10,16.62,0.33,7.01,3.09,1.08,14.13,0.01,14.11,0.00,14.11,14.11,0.00
1,060310,3S,2020/03,225.92,188.78,37.14,3.64,8.95,4.67,0.21,7.73,14.01,-3.89,-10.74,0.07,-10.80,0.00,-10.80,-10.80,0.00
2,060310,3S,2021/03,234.78,220.02,14.75,3.14,5.65,-14.08,0.13,34.36,4.87,-3.53,7.39,0.44,6.95,0.00,6.95,6.95,0.00
3,060310,3S,2021/12,180.62,137.21,43.41,2.11,5.02,20.22,0.08,3.60,0.18,0.00,22.03,0.62,21.42,0.00,21.42,19.73,1.69
4,095570,AJ네트웍스,2018/12,10455.30,0.00,10455.30,64.01,642.42,-212.36,42.94,157.22,89.77,8.07,-494.08,62.45,-556.54,929.23,372.69,11.33,361.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9339,037440,희림,2021/12,2120.92,1870.44,250.48,6.30,36.78,82.84,14.00,6.82,0.70,-0.96,80.75,18.94,61.82,0.00,61.82,61.82,-0.00
9340,238490,힘스,2018/12,354.55,253.11,101.44,4.63,26.08,-17.09,11.87,0.88,1.23,-0.64,-17.44,-7.22,-10.22,0.00,-10.22,0.00,0.00
9341,238490,힘스,2019/12,850.65,566.99,283.66,54.79,42.13,77.02,31.85,2.32,5.84,-1.21,79.38,-5.04,84.42,0.00,84.42,84.42,0.00
9342,238490,힘스,2020/12,1062.06,604.84,457.22,31.91,54.84,249.68,21.36,0.54,9.78,0.08,202.40,30.35,172.05,0.00,172.05,172.05,0.00


In [17]:
# DB 데이터 저장 확인
sql = 'select * from stock_con_q'
db_data = pd.read_sql(sql, engine)
db_data

,종목코드,종목명,기간,매출액,매출원가,매출총이익,판매비,관리비,영업이익,금융수익,금융원가,기타수익,기타비용,관계기업관련손익,EBIT,법인세비용,계속영업이익,중단영업이익,당기순이익
0,060310,3S,2021/03,37.43,40.70,-3.28,-1.08,-3.22,-7.42,0.04,1.14,29.65,0.44,-3.53,17.16,0.33,16.82,0.00,16.82
1,060310,3S,2021/06,37.76,32.66,5.10,0.78,2.09,-2.82,0.03,0.49,0.36,0.11,0.00,-3.03,0.60,-3.63,0.00,-3.63
2,060310,3S,2021/09,76.44,55.76,20.69,0.80,1.29,13.38,0.02,0.59,2.05,0.02,0.00,14.84,0.02,14.83,0.00,14.83
3,060310,3S,2021/12,66.42,48.80,17.62,1.54,6.42,9.66,0.04,0.62,1.20,0.06,0.00,10.22,0.00,10.22,0.00,10.22
4,095570,AJ네트웍스,2021/03,2561.41,0.00,2561.41,164.52,175.44,45.30,17.88,88.90,14.09,3.01,-14.80,-29.45,13.73,-43.19,90.98,47.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9339,037440,희림,2021/12,624.55,535.07,89.48,6.30,36.78,47.94,1.86,4.40,6.17,0.41,-0.83,50.33,11.91,38.41,0.00,38.41
9340,238490,힘스,2021/03,105.64,82.81,22.83,7.73,3.93,-18.82,15.62,0.23,0.43,0.06,0.00,-3.06,-1.59,-1.47,0.00,-1.47
9341,238490,힘스,2021/06,144.19,118.00,26.19,-6.03,0.00,-7.22,10.40,10.08,0.18,0.08,0.03,-6.77,-2.89,-3.88,0.00,-3.88
9342,238490,힘스,2021/09,206.73,115.86,90.87,-5.36,0.00,51.25,14.34,2.39,0.17,0.03,-0.00,63.33,10.93,52.40,0.00,52.40


In [18]:
# DB 데이터 저장 확인
sql = 'select * from stock_uncon_a'
db_data = pd.read_sql(sql, engine)
db_data

,종목코드,종목명,기간,매출액,매출원가,매출총이익,판매비,관리비,영업이익,금융수익,...,기타수익,기타비용,관계기업관련손익,EBIT,법인세비용,계속영업이익,중단영업이익,당기순이익,지배주주순이익,비지배주주순이익
0,060310,3S,2019/03,319.51,273.48,46.03,7.04,8.85,15.30,0.33,...,7.01,3.01,0.00,11.83,-0.02,11.85,0.0,11.85,None,None
1,060310,3S,2020/03,194.70,162.95,31.75,3.62,7.81,3.89,0.20,...,7.72,13.82,0.00,-7.23,0.05,-7.28,0.0,-7.28,None,None
2,060310,3S,2021/03,220.00,209.57,10.42,3.14,4.72,-14.20,0.10,...,32.95,6.33,0.00,7.96,0.41,7.54,0.0,7.54,None,None
3,060310,3S,2021/12,149.86,115.58,34.27,2.08,3.21,16.96,0.08,...,3.58,0.16,0.00,18.79,0.60,18.19,0.0,18.19,None,None
4,095570,AJ네트웍스,2018/12,3398.00,0.00,3398.00,10.97,132.31,373.38,72.40,...,58.37,97.52,-204.34,-170.09,53.30,-223.40,0.0,-223.40,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9339,037440,희림,2021/12,2118.29,1870.30,247.99,8.50,36.23,80.31,13.99,...,2.46,2.14,0.00,73.38,18.94,54.44,0.0,54.44,None,None
9340,238490,힘스,2018/12,354.55,253.11,101.44,4.63,26.08,-17.09,11.87,...,0.88,1.23,-0.64,-17.44,-7.22,-10.22,0.0,-10.22,None,None
9341,238490,힘스,2019/12,850.65,566.99,283.66,54.79,42.13,77.02,31.85,...,2.32,5.84,-1.21,79.38,-5.04,84.42,0.0,84.42,None,None
9342,238490,힘스,2020/12,1062.06,604.84,457.22,31.91,57.21,248.02,21.35,...,0.54,9.78,1.66,202.32,30.27,172.05,0.0,172.05,None,None


In [19]:
# DB 데이터 저장 확인
sql = 'select * from stock_uncon_q'
db_data = pd.read_sql(sql, engine)
db_data

,종목코드,종목명,기간,매출액,매출원가,매출총이익,판매비,관리비,영업이익,금융수익,금융원가,기타수익,기타비용,관계기업관련손익,EBIT,법인세비용,계속영업이익,중단영업이익,당기순이익
0,060310,3S,2021/03,35.44,38.37,-2.93,-0.95,-2.58,-6.86,0.01,1.13,28.32,2.11,0.00,18.22,0.41,17.81,0.0,17.81
1,060310,3S,2021/06,37.28,33.33,3.95,0.78,1.58,-2.10,0.02,0.49,0.34,0.11,0.00,-2.33,0.60,-2.93,0.0,-2.93
2,060310,3S,2021/09,55.77,42.12,13.65,0.78,0.56,8.51,0.02,0.57,2.05,0.02,0.00,9.99,0.00,9.99,0.0,9.99
3,060310,3S,2021/12,56.81,40.14,16.67,1.43,4.69,10.55,0.04,0.61,1.19,0.04,0.00,11.13,0.00,11.13,0.0,11.13
4,095570,AJ네트웍스,2021/03,1279.02,0.00,1279.02,67.44,200.25,65.67,20.68,77.53,136.47,29.61,-58.01,57.66,35.56,22.11,0.0,22.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9339,037440,희림,2021/12,623.94,535.02,88.93,8.50,36.23,45.39,1.86,4.40,1.92,1.89,0.00,42.88,11.91,30.96,0.0,30.96
9340,238490,힘스,2021/03,105.64,82.81,22.83,7.73,3.41,-17.49,15.62,0.23,0.43,1.39,0.00,-3.06,-1.59,-1.47,0.0,-1.47
9341,238490,힘스,2021/06,144.19,118.00,26.19,-6.03,0.00,-7.58,10.40,10.06,0.18,0.08,0.37,-6.77,-2.89,-3.88,0.0,-3.88
9342,238490,힘스,2021/09,206.73,115.86,90.87,-5.36,0.00,51.30,14.34,2.39,0.17,0.03,-0.06,63.33,10.93,52.40,0.0,52.40


- 9344 rows 출력 완료

In [20]:
engine.dispose() # 연결 해제